In [139]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, ReLU, Conv2D

from tensorflow.nn import conv2d, conv2d_transpose
from tensorflow.io import read_file
from tensorflow.image import decode_image
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sys

### Helpers

In [2]:
def _conv2d(x, kernel):
    return conv2d(x, kernel, strides=[1, 2, 2, 1], padding='SAME')

def _conv2d_transpose(x, kernel, output_shape):
    return conv2d_transpose(
            x, kernel,
            output_shape=output_shape,
            strides=[1, 2, 2, 1],
            padding='SAME')

def load_image(image_path):
    image = read_file(image_path)
    image = decode_image(image)

    return image

In [3]:
class WaveletPooling(Layer):
    """
    Wavelet Pooing Custom Layer
    """
    def __init__(self, name=''):
        super(WaveletPooling, self).__init__()
        self._name = name
        square_of_2 = tf.math.sqrt(tf.constant(2, dtype=tf.float32))
        L = tf.math.divide(
            tf.constant(1, dtype=tf.float32),
            tf.math.multiply(square_of_2, tf.constant([[1, 1]], dtype=tf.float32))
        )
        H = tf.math.divide(
            tf.constant(1, dtype=tf.float32),
            tf.math.multiply(square_of_2, tf.constant([[-1, 1]], dtype=tf.float32))
        )

        self.LL = tf.reshape(tf.math.multiply(tf.transpose(L), L), (1, 2, 2, 1))
        self.LH = tf.reshape(tf.math.multiply(tf.transpose(L), H), (1, 2, 2, 1))
        self.HL = tf.reshape(tf.math.multiply(tf.transpose(H), L), (1, 2, 2, 1))
        self.HH = tf.reshape(tf.math.multiply(tf.transpose(H), H), (1, 2, 2, 1))

    def call(self, inputs):
        LL, LH, HL, HH = self.repeat_filters(inputs.shape[-1])
        return [_conv2d(inputs, LL),
                _conv2d(inputs, LH),
                _conv2d(inputs, HL),
                _conv2d(inputs, HH)]

    def compute_output_shape(self, input_shape):
        shape = (
            input_shape[0], input_shape[1] // 2,
            input_shape[2] // 2, input_shape[3]
        )

        return [shape, shape, shape, shape]

    def repeat_filters(self, repeats):
        # Can we optimize this?
        return [
            tf.transpose(tf.repeat(self.LL, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.LH, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.HL, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.HH, repeats, axis=0), (1, 2, 3, 0))
        ]

class WaveletUnpooling(Layer):
    """
    Wavelet Unpooing Custom Layer
    """
    def __init__(self, name):
        super(WaveletUnpooling, self).__init__()
        self._name = name
        square_of_2 = tf.math.sqrt(tf.constant(2, dtype=tf.float32))
        L = tf.math.divide(
            tf.constant(1, dtype=tf.float32),
            tf.math.multiply(square_of_2, tf.constant([[1, 1]], dtype=tf.float32))
        )
        H = tf.math.divide(
            tf.constant(1, dtype=tf.float32),
            tf.math.multiply(square_of_2, tf.constant([[-1, 1]], dtype=tf.float32))
        )

        self.LL = tf.reshape(tf.math.multiply(tf.transpose(L), L), (1, 2, 2, 1))
        self.LH = tf.reshape(tf.math.multiply(tf.transpose(L), H), (1, 2, 2, 1))
        self.HL = tf.reshape(tf.math.multiply(tf.transpose(H), L), (1, 2, 2, 1))
        self.HH = tf.reshape(tf.math.multiply(tf.transpose(H), H), (1, 2, 2, 1))

    def call(self, inputs):
        LL_in, LH_in, HL_in, HH_in, tensor_in = inputs
        LL, LH, HL, HH = self.repeat_filters(LL_in.shape[-1])
        out_shape = tf.shape(tensor_in)

        return _conv2d_transpose(LL_in, LL, output_shape=out_shape) + _conv2d_transpose(LH_in, LH, output_shape=out_shape) + _conv2d_transpose(HL_in, HL, output_shape=out_shape) + _conv2d_transpose(HH_in, HH, output_shape=out_shape)

    def compute_output_shape(self, input_shape):
        _ip_shape = input_shape[0]
        shape = (
            _ip_shape[0],
            _ip_shape[1] * 2,
            _ip_shape[2] * 2,
            sum(ips[3] for ips in input_shape)
        )

        return shape

    def repeat_filters(self, repeats):
        # Can we optimize this?
        return [
            tf.transpose(tf.repeat(self.LL, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.LH, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.HL, repeats, axis=0), (1, 2, 3, 0)),
            tf.transpose(tf.repeat(self.HH, repeats, axis=0), (1, 2, 3, 0)),
        ]

class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [tf.keras.layers.InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        if s[1] == None:
            return (None, None, None, s[3])
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad, h_pad = self.padding
        return tf.pad(x, [[0, 0], [h_pad, h_pad], [w_pad, w_pad], [0, 0]], 'REFLECT')

    def get_config(self):
        config = super(ReflectionPadding2D, self).get_config()
        print(config)
        return config

class CNNBlock(Layer):
    def __init__(self, filters, kernel, name):
        super(CNNBlock, self).__init__()
        self._name = name
        
        self.padding = ReflectionPadding2D()
        self.conv2d = Conv2D(filters, kernel, padding = 'valid')
        self.activation = ReLU()

    def call(self, inputs):
        x = self.padding(inputs)
        x = self.conv2d(x)
        x = self.activation(x)
        
        return x

In [4]:
def preprocess_feat(feat, center = False):
    feat = tf.reshape(feat, (-1, feat.shape[-1]))
    feat = tf.transpose(feat)
    
    feat_mean = tf.math.reduce_mean(feat, 1)
    
    if center:
        feat_mean = tf.expand_dims(feat_mean, 1)
        feat = tf.subtract(feat, feat_mean)
        
    return feat,feat_mean

def center_feat(feat):
    feat_mean = tf.math.reduce_mean(feat, 1)
    feat_mean = tf.expand_dims(feat_mean, 1)
    return tf.subtract(feat, feat_mean)

def get_svd(feat):
    feat = center_feat(feat)
    feat = tf.matmul(feat, feat, transpose_b=True) / (feat.shape[1] - 1)
    feat = feat + tf.eye(feat.shape[0])
    
    return tf.linalg.svd(feat)

def get_style_correlation_transform(feat, return_mean = False):
    feat, mean = preprocess_feat(feat)
    s_e, _, s_v = get_svd(feat)
    # The inverse of the content singular values matrix operation (^-0.5)
    s_e = tf.pow(s_e, 0.5)
    
    EDE = tf.matmul(tf.matmul(s_v, tf.linalg.diag(s_e)), s_v, transpose_b=True)
    return (EDE, mean) if return_mean else EDE

def wct(content_feat_raw, style_feat_raw, alpha = 1):
    style_EDE, style_mean = get_style_correlation_transform(style_feat_raw, return_mean = True)
    content_feat, content_mean = preprocess_feat(content_feat_raw, center=True)
    
    c_e, _, c_v = get_svd(content_feat)
    c_e = tf.pow(c_e, -0.5)
    
    content_EDE = tf.matmul(tf.matmul(c_v, tf.linalg.diag(c_e)), c_v, transpose_b=True)
    content_whitened = tf.matmul(content_EDE, content_feat)

    final_out = tf.matmul(style_EDE, content_whitened)
    final_out = tf.add(final_out, tf.expand_dims(style_mean, 1))
    final_out = tf.clip_by_value(
        final_out,
        tf.math.reduce_min(content_feat),
        tf.math.reduce_max(content_feat),
    )
    
    final_out = tf.reshape(tf.transpose(final_out), content_feat_raw.shape)
    final_out = alpha * final_out + (1 - alpha) * content_feat_raw
    return final_out

In [5]:
class WaveletEncoder(Model):
    def __init__(self):
        super().__init__()
        self._name = 'WaveletEncoder'
        
        self.preprocessing_conv2d = Conv2D(3, 1, padding = 'valid')
        
        ### Block 1
        self.block1_conv2d_1 = CNNBlock(64, 3, 'WE_block1_conv2d_1')
        self.block1_conv2d_2 = CNNBlock(64, 3, 'WE_block1_conv2d_2')
        self.block1_pooling = WaveletPooling('WE_block1_pooling')
        
        ### Block 2
        self.block2_conv2d_1 = CNNBlock(128, 3, 'WE_block2_conv2d_1')
        self.block2_conv2d_2 = CNNBlock(128, 3, 'WE_block2_conv2d_2')
        self.block2_pooling = WaveletPooling('WE_block2_pooling')
        
        ### Block 3
        self.block3_conv2d_1 = CNNBlock(256, 3, 'WE_block3_conv2d_1')
        self.block3_conv2d_2 = CNNBlock(256, 3, 'WE_block3_conv2d_2')
        self.block3_conv2d_3 = CNNBlock(256, 3, 'WE_block3_conv2d_3')
        self.block3_conv2d_4 = CNNBlock(256, 3, 'WE_block3_conv2d_4')
        self.block3_pooling = WaveletPooling('WE_block3_pooling')
        
        ### Block 4
        self.block4_conv2d_1 = CNNBlock(512, 3, 'WE_block4_conv2d_1')

    def call(self, inputs, style_feat = None, trainable = False):
        wavelet_skips = {
            'block1': None,
            'block2': None,
            'block3': None,
        }
        
        features = {
            'block1': None,
            'block2': None,
            'block3': None
        }
        
        x = self.preprocessing_conv2d(inputs)
        
        x = self.block1_conv2d_1(x)
        x = self.block1_conv2d_2(x)
        LL_1, LH_1, HL_1, HH_1 = self.block1_pooling(x)
        wavelet_skips['block1'] = [LH_1, HL_1, HH_1, x]
        features['block1'] = LL_1
        
        if style_feat:
            LL_1 = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (LL_1, style_feat['block1']),
                dtype=LL_1.dtype
            )
            
        x = self.block2_conv2d_1(LL_1)
        x = self.block2_conv2d_2(x)
        LL_2, LH_2, HL_2, HH_2 = self.block2_pooling(x)
        wavelet_skips['block2'] = [LH_2, HL_2, HH_2, x]
        features['block2'] = LL_2
        
        if style_feat:
            LL_2 = tf.map_fn(
                lambda y: wct(y[0], y[1]),
                (LL_2, style_feat['block2']),
                dtype=LL_2.dtype
            )
            
        x = self.block3_conv2d_1(LL_2)
        x = self.block3_conv2d_2(x)
        x = self.block3_conv2d_3(x)
        x = self.block3_conv2d_4(x)
        LL_3, LH_3, HL_3, HH_3 = self.block3_pooling(x)
        wavelet_skips['block3'] = [LH_3, HL_3, HH_3, x]
        features['block3'] = LL_3
        
        if style_feat:
            LL_3 = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (LL_3, style_feat['block3']),
                dtype=LL_3.dtype
            )
        
        x = self.block4_conv2d_1(LL_3)
        features['block4'] = x
        
        if style_feat:
            x = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (x, style_feat['block4']),
                dtype=x.dtype
            )
            
        return x, wavelet_skips, features
    
class WaveletDecoder(Model):
    def __init__(self):
        super().__init__()
        self._name = 'WaveletDecoder'

        ### Block 3
        self.block3_conv2d_1 = CNNBlock(256, 3, 'WD_block4_conv2d_1')
        self.block3_unpooling = WaveletUnpooling('WD_block4_unpooling')
        self.block3_conv2d_2 = CNNBlock(256, 3, 'WD_block3_conv2d_2')
        self.block3_conv2d_3 = CNNBlock(256, 3, 'WD_block3_conv2d_3')
        self.block3_conv2d_4 = CNNBlock(256, 3, 'WD_block3_conv2d_4')

        ### Block 2
        self.block2_conv2d_1 = CNNBlock(128, 3, 'WE_block2_conv2d_1')
        self.block2_unpooling = WaveletUnpooling('WE_block2_unpooling')
        self.block2_conv2d_2 = CNNBlock(128, 3, 'WE_block2_conv2d_2')

        ### Block 1
        self.block1_conv2d_1 = CNNBlock(64, 3, 'WE_block1_conv2d_1')
        self.block1_unpooling = WaveletUnpooling('WE_block1_unpooling')
        self.block1_conv2d_2 = CNNBlock(64, 3, 'WE_block1_conv2d_2')

        self.post_processing_padding = ReflectionPadding2D()
        self.post_processing_conv2d = Conv2D(3, 3, padding = 'valid')

    def call(self, inputs, skips, style_feat = None, trainable = False):
        features = {
            'block3': None,
            'block2': None,
            'block1': None,
        }
        
        x = self.block3_conv2d_1(inputs)
        x = self.block3_unpooling([x, *skips['block3']])
        x = self.block3_conv2d_2(x)
        x = self.block3_conv2d_3(x)
        x = self.block3_conv2d_4(x)
        features['block3'] = x
        
        if style_feat:
            x = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (x, style_feat['block3']),
                dtype=x.dtype
            )
        
        x = self.block2_conv2d_1(x)
        x = self.block2_unpooling([x, *skips['block2']])
        x = self.block2_conv2d_2(x)
        features['block2'] = x

        if style_feat:
            x = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (x, style_feat['block2']),
                dtype=x.dtype
            )
            
        x = self.block1_conv2d_1(x)
        x = self.block1_unpooling([x, *skips['block1']])
        x = self.block1_conv2d_2(x)
        features['block1'] = x
        
        if style_feat:
            x = tf.map_fn(
                lambda x: wct(x[0], x[1]),
                (x, style_feat['block1']),
                dtype=x.dtype
            )
            
        x = self.post_processing_padding(x)
        x = self.post_processing_conv2d(x)

        return x, features

In [6]:
class WaveletAE(Model):
    def __init__(self):
        super().__init__()
        self._name = 'WaveletAE'
        self.encoder = WaveletEncoder()
        self.decoder = WaveletDecoder()
        self.load_weights('../weights/wavelet_autoencoder')

    def call(self, inputs, trainable = False):
        x, skips, _ = self.encoder(inputs)
        output, _ = self.decoder(x, skips)

        return output
        
    def transfer(self, content_img, style_img, encoder_transfer = True, skips_transfer = True, decoder_transfer = True, alpha = 1):
        style_features, style_skips = self.get_features(style_img)
        x, content_skips, _ = self.encoder(content_img, style_features['encoder'] if encoder_transfer else None)
        
        if skips_transfer:
            for key in content_skips.keys():
                for i in range(3):
                    content_skips[key][0] = tf.map_fn(
                        lambda x: wct(x[0], x[1]),
                        (content_skips[key][0], style_skips[key][0]),
                        dtype=tf.float32
                    )
                
        out, _ = self.decoder(x, content_skips, style_features['decoder'] if decoder_transfer else None)
        out = tf.clip_by_value(out, 0, 1)
        
        return out
    
    def get_features(self, inputs):
        encoder_out, skips, encoder_feat = self.encoder(inputs)
        _, decoder_feat = self.decoder(encoder_out, skips)
        
        features = {
            'encoder': encoder_feat,
            'decoder': decoder_feat,
        }
        
        return features, skips
        
test_img = tf.io.read_file('../assets/moli_content.jpg')
test_img = tf.image.decode_image(test_img, dtype=tf.float16)

test_img_style = tf.io.read_file('../assets/moli_style.jpg')
test_img_style = tf.image.decode_image(test_img_style, dtype=tf.float16)

wavelet_ae = WaveletAE()

In [112]:
def sample_from_corr_matrix(sigma, num_features = None, eigenvalues = None, eigenvectors = None):
    data = tf.eye(sigma.shape[0], num_features)

    if not eigenvectors or not eigenvectors:
        eigenvalues, u, eigenvectors = tf.linalg.svd(sigma)
        eigenvalues = tf.linalg.diag(eigenvalues)
        
    data = tf.matmul(tf.matmul(eigenvectors, tf.sqrt(eigenvalues)), data)
    
    return data

In [115]:
test = tf.random.uniform((10, 10))
test

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[0.81814516, 0.67093635, 0.9114053 , 0.01255274, 0.60665643,
        0.87863755, 0.10737753, 0.61788845, 0.7887709 , 0.37638962],
       [0.5569432 , 0.9119735 , 0.03203976, 0.64768994, 0.6504371 ,
        0.88924754, 0.9991813 , 0.8690746 , 0.06949031, 0.91188097],
       [0.16054773, 0.8682692 , 0.3797729 , 0.19993234, 0.7577106 ,
        0.02513528, 0.2583996 , 0.10814404, 0.48027182, 0.11917496],
       [0.41369414, 0.7429981 , 0.9240122 , 0.60481   , 0.08651209,
        0.9741082 , 0.7406219 , 0.27923965, 0.08161557, 0.8272071 ],
       [0.79215777, 0.37163877, 0.20451856, 0.8862821 , 0.5812131 ,
        0.9042901 , 0.22144556, 0.0365181 , 0.86242425, 0.20869052],
       [0.7541809 , 0.63204896, 0.731931  , 0.5966114 , 0.38791597,
        0.07000744, 0.1448189 , 0.1772914 , 0.37726223, 0.93862474],
       [0.9329052 , 0.03799462, 0.01893723, 0.17572784, 0.29898608,
        0.26945972, 0.94430697, 0.03457725, 0.98951364, 0.41

In [176]:
e, u, v = tf.linalg.svd(test)
e = pad_sequences(tf.expand_dims(e[:5], 0), maxlen=e.shape[0], padding='post', dtype='float32')[0]
e = tf.linalg.diag(e)
e

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[5.1148043, 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       ],
       [0.       , 1.6161581, 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 1.2747265, 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 1.0543399, 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.9879097, 0.       ,
        0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       ],
       [0.       , 0.       , 0.       , 0.       , 0.       

In [177]:
tf.matmul(tf.matmul(u, e), v, transpose_b=True)

<tf.Tensor: shape=(10, 10), dtype=float32, numpy=
array([[ 0.7414155 ,  0.8422563 ,  0.865973  ,  0.14717644,  0.6085492 ,
         0.89890456,  0.1602135 ,  0.4680623 ,  0.80021733,  0.21415482],
       [ 0.40664363,  0.8301804 ,  0.15668148,  0.6551702 ,  0.65002227,
         0.91718006,  1.0986443 ,  0.8425497 ,  0.12763266,  0.89500344],
       [ 0.26298672,  0.6851393 ,  0.4660395 ,  0.04865327,  0.60821164,
        -0.04073204,  0.24273486,  0.3852963 ,  0.5010355 ,  0.2695733 ],
       [ 0.5419282 ,  0.7246532 ,  0.7124187 ,  0.77587014,  0.2714573 ,
         0.95961887,  0.43242183,  0.43172425,  0.02956969,  0.8205058 ],
       [ 0.92853206,  0.26814646,  0.40736395,  0.535882  ,  0.310881  ,
         0.833755  ,  0.45031723,  0.12816715,  0.83230746,  0.36940756],
       [ 0.6008828 ,  0.64543796,  0.8199284 ,  0.72891223,  0.37951854,
         0.08356208,  0.2536218 ,  0.13174513,  0.42454335,  0.7783868 ],
       [ 0.8344478 ,  0.03990941, -0.09182064,  0.4330673 ,  0.44778